# Notes from PB Python's group-aggregate

[Comprehensive Guide to Grouping and Aggregating with Pandas](https://pbpython.com/groupby-agg.html)

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

# Titanic dataset

In [2]:
df = sns.load_dataset('titanic')
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


# Aggregation

>"Aggregation: takes multiple individual values and returns a summary"

In [3]:
# agg of a single column
df['fare'].agg(['sum', 'mean'])

sum     28693.949300
mean       32.204208
Name: fare, dtype: float64

In [4]:
# agg of multiple columns
df[['fare', 'age']].agg(['sum', 'mean'])

,fare,age
sum,28693.949300,21205.170000
mean,32.204208,29.699118


**Named aggregrations**

You can have different set of agg functions on different columns. Three ways of doing this
- List: What we did above
- Dict (recommended):
- Tuple: one agg per tuple; assign name to output

In [5]:
# dict
df.agg({'fare':['sum', 'mean'], 'sex':['count']})

,fare,sex
sum,28693.949300,NaN
mean,32.204208,NaN
count,NaN,891.0


In [6]:
# tuple
# df.agg(x=('fare', max), y=('fare', 'min'), z=('C', np.mean))

# Groupby

### Basic match

In [7]:
agg_func_math = {'fare': ['sum', 'mean']}
df.groupby(by=['embark_town']).agg(agg_func_math).round(2)

fare       
                  sum   mean
embark_town                 
Cherbourg    10072.30  59.95
Queenstown    1022.25  13.28
Southampton  17439.40  27.08

In [8]:
agg_func_math = {'fare':['describe']}
df.groupby(by=['embark_town']).agg(agg_func_math)

fare                                                 \
            describe                                                  
               count       mean        std     min       25%    50%   
embark_town                                                           
Cherbourg      168.0  59.954144  83.912994  4.0125  13.69795  29.70   
Queenstown      77.0  13.276030  14.188047  6.7500   7.75000   7.75   
Southampton    644.0  27.079812  35.887993  0.0000   8.05000  13.00   

                                  
                                  
                   75%       max  
embark_town                       
Cherbourg    78.500025  512.3292  
Queenstown   15.500000   90.0000  
Southampton  27.900000  263.0000

### Counting

In [9]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [10]:
agg_func_count = {'embark_town':['count', 'nunique', 'size']}
# df.groupby(by=['deck']).agg(agg_func_count)
df.agg(agg_func_count)

,embark_town
count,889
nunique,3
size,891


### First & Last

In [11]:
agg_func_dict = {'fare': ['first', 'last']}
df.sort_values(by='fare',ascending=False).groupby(by=['embark_town']).agg(agg_func_dict)

fare        
                first    last
embark_town                  
Cherbourg    512.3292  4.0125
Queenstown    90.0000  6.7500
Southampton  263.0000  0.0000

In [12]:
#idxmin,max
agg_func_dict = {'fare': ['idxmin', 'idxmax']}
df.groupby(by=['embark_town']).agg(agg_func_dict)

fare       
            idxmin idxmax
embark_town              
Cherbourg      378    258
Queenstown     143    245
Southampton    179     27

In [13]:
df.loc[[378, 258]]

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
378,0,3,male,20.0,0,0,4.0125,C,Third,man,True,NaN,Cherbourg,no,True
258,1,1,female,35.0,0,0,512.3292,C,First,woman,False,NaN,Cherbourg,yes,True


In [14]:
# see rows with max value

df.loc[df.groupby(by='class')['fare'].idxmax()]

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
258,1,1,female,35.0,0,0,512.3292,C,First,woman,False,NaN,Cherbourg,yes,True
72,0,2,male,21.0,0,0,73.5000,S,Second,man,True,NaN,Southampton,no,True
159,0,3,male,NaN,8,2,69.5500,S,Third,man,True,NaN,Southampton,no,False


### Other libraries

In [15]:
from scipy.stats import skew, mode
agg_func_stats = {'fare': [skew, mode, pd.Series.mode]}

df.groupby(by='embark_town').agg(agg_func_stats)

fare                          
                 skew              mode    mode
embark_town                                    
Cherbourg    3.305112  ([7.2292], [15])  7.2292
Queenstown   4.265111    ([7.75], [30])  7.7500
Southampton  3.640276    ([8.05], [43])  8.0500

### Working with text

In [16]:
agg_func_text = {'deck': ['nunique', mode, set]}
df.groupby(by=['class']).agg(agg_func_text)

deck                                   
       nunique         mode                   set
class                                            
First        5  ([C], [59])  {nan, C, A, D, B, E}
Second       3   ([F], [8])        {nan, D, F, E}
Third        3   ([F], [5])        {nan, G, F, E}

# Custom functions

In [17]:
from functools import partial

In [18]:
q_25 = partial(pd.Series.quantile, q=0.25)
q_25.__name__ = '%25'

In [19]:
def percentile_25(x):
    return x.quantile(.25)

In [20]:
lambda_25 = lambda x:x.quantile(.25)
lambda_25.__name__ = 'lambda_25%'

In [21]:
agg_func = {'fare':[q_25, percentile_25, lambda_25, lambda x:x.quantile(.25)]}
df.groupby(by=['embark_town']).agg(agg_func).round(2)

fare                                    
               %25 percentile_25 lambda_25% <lambda_0>
embark_town                                           
Cherbourg    13.70         13.70      13.70      13.70
Queenstown    7.75          7.75       7.75       7.75
Southampton   8.05          8.05       8.05       8.05

#### Custom function examples

In [22]:
def count_nulls(s):
    return s.size - s.count()

def count_nulls_ur(s):
    return s.isnull().sum()

def unique_nan(s):
    return s.nunique(dropna=False)

agg_func_custom_count = {'embark_town':['count','size',unique_nan, count_nulls,count_nulls_ur, set]}
df.groupby(['deck']).agg(agg_func_custom_count)

embark_town                                             \
           count size unique_nan count_nulls count_nulls_ur   
deck                                                          
A             15   15          2           0              0   
B             45   47          3           2              2   
C             59   59          3           0              0   
D             33   33          2           0              0   
E             32   32          3           0              0   
F             13   13          3           0              0   
G              4    4          1           0              0   

                                            
                                       set  
deck                                        
A                 {Cherbourg, Southampton}  
B            {nan, Cherbourg, Southampton}  
C     {Queenstown, Cherbourg, Southampton}  
D                 {Cherbourg, Southampton}  
E     {Queenstown, Cherbourg, Southampton}  
F     {Queenstown, Cherbourg, Southampton}  
G                            {Southampton}

In [23]:
from scipy.stats import trim_mean

def percentile_90(x):
    return x.quantile(.9)

def trim_mean_10(x):
    return trim_mean(x, 0.1)

def largest(x):
    return x.nlargest(1)

agg_func_largest = {'fare':[percentile_90, trim_mean_10, largest]}
df.groupby(['class', 'embark_town']).agg(agg_func_largest)

fare                       
                   percentile_90 trim_mean_10   largest
class  embark_town                                     
First  Cherbourg        227.5250    85.408335  512.3292
       Queenstown        90.0000    90.000000   90.0000
       Southampton      152.3150    60.500160  263.0000
Second Cherbourg         41.5792    25.167500   41.5792
       Queenstown        12.3500    12.350000   12.3500
       Southampton       31.7500    18.202273   73.5000
Third  Cherbourg         19.0229    10.677941   22.3583
       Queenstown        24.0600     9.670476   29.1250
       Southampton       31.2750    11.501469   69.5500

In [24]:
def top_10_sum(x):
    return x.nlargest(10).sum()
    
def bottom_10_sum(x):
    return x.nsmallest(10).sum()
    
agg_fun_top_bottom_sum = {'fare':[top_10_sum, bottom_10_sum]}    
df.groupby('class').agg(agg_fun_top_bottom_sum)

fare              
       top_10_sum bottom_10_sum
class                          
First   3361.2584      108.3709
Second   622.2376       42.0000
Third    656.3374       36.1291

# Custom functions w/ multiple columns

In [25]:
def summary(x):
    result = {
        'fare_sum': x['fare'].sum(),
        'fare_mean': x['fare'].mean(),
        'fare_range': x['fare'].max() - x['fare'].min()
    }
    return pd.Series(result).round(0)

df.groupby(['class']).apply(summary)

,fare_sum,fare_mean,fare_range
class,,,
First,18177.0,84.0,512.0
Second,3802.0,21.0,74.0
Third,6715.0,14.0,70.0


### Working with Group Objects

In [26]:
tmp_df = df.groupby(by=['embark_town', 'class']).agg({'fare':'sum'})
tmp_df

fare
embark_town class            
Cherbourg   First   8901.0750
            Second   431.0917
            Third    740.1295
Queenstown  First    180.0000
            Second    37.0500
            Third    805.2043
Southampton First   8936.3375
            Second  3333.7000
            Third   5169.3613

In [27]:
tmp_df.assign(pct_total=lambda x: x/x.sum())

fare  pct_total
embark_town class                       
Cherbourg   First   8901.0750   0.311947
            Second   431.0917   0.015108
            Third    740.1295   0.025939
Queenstown  First    180.0000   0.006308
            Second    37.0500   0.001298
            Third    805.2043   0.028219
Southampton First   8936.3375   0.313183
            Second  3333.7000   0.116833
            Third   5169.3613   0.181165

In [28]:
# same as above w/ crosstab

pd.crosstab(df['embark_town'],df['class'], values=df['fare'], aggfunc='sum', normalize=True)

class,First,Second,Third
embark_town,,,
Cherbourg,0.311947,0.015108,0.025939
Queenstown,0.006308,0.001298,0.028219
Southampton,0.313183,0.116833,0.181165


In [29]:
# agg with pivot tables
pd.pivot_table(data=df, index=['embark_town'], columns=['class'], aggfunc=agg_fun_top_bottom_sum)

fare                                                  
            bottom_10_sum                    top_10_sum                    
class               First    Second    Third      First    Second     Third
embark_town                                                                
Cherbourg        282.9957  172.2041  68.2500  3239.3542  334.6084  196.7457
Queenstown       180.0000   37.0500  73.5916   180.0000   37.0500  264.5750
Southampton      108.3709   42.0000  39.6291  2237.5251  614.5000  656.3374

In [30]:
# multiple groupby

In [31]:
# fare_group = df.groupby(['embark_town', 'class']).agg({'fare': 'sum'})
fare_group = df.groupby(by=['embark_town', 'class']).agg({'fare':'sum'})
fare_group

fare
embark_town class            
Cherbourg   First   8901.0750
            Second   431.0917
            Third    740.1295
Queenstown  First    180.0000
            Second    37.0500
            Third    805.2043
Southampton First   8936.3375
            Second  3333.7000
            Third   5169.3613

In [32]:
fare_group.groupby(level=0).cumsum()

fare
embark_town class             
Cherbourg   First    8901.0750
            Second   9332.1667
            Third   10072.2962
Queenstown  First     180.0000
            Second    217.0500
            Third    1022.2543
Southampton First    8936.3375
            Second  12270.0375
            Third   17439.3988

In [33]:
# Grouper

sales = pd.read_excel('https://github.com/chris1610/pbpython/blob/master/data/2018_Sales_Total_v2.xlsx?raw=True')

daily_sales = sales.groupby([pd.Grouper(key='date', freq='D')]).agg(daily_sales=('ext price','sum')).reset_index()
daily_sales['quarter_sales'] = daily_sales.groupby(pd.Grouper(key='date', freq='Q')).agg({'daily_sales':'cumsum'})

In [34]:
daily_sales[daily_sales['date'].between('2018-3-29', '2018-4-4')]

,date,daily_sales,quarter_sales
87,2018-03-29,5112.17,509316.79
88,2018-03-30,7212.31,516529.10
89,2018-03-31,10300.89,526829.99
90,2018-04-01,4550.97,4550.97
91,2018-04-02,4337.54,8888.51
92,2018-04-03,3731.16,12619.67
93,2018-04-04,8110.77,20730.44


# Flatten Heirarchical Column Indices

In [35]:
df.groupby(['embark_town', 'class']).agg({'fare': ['sum', 'mean']}).round(0)

fare       
                       sum   mean
embark_town class                
Cherbourg   First   8901.0  105.0
            Second   431.0   25.0
            Third    740.0   11.0
Queenstown  First    180.0   90.0
            Second    37.0   12.0
            Third    805.0   11.0
Southampton First   8936.0   70.0
            Second  3334.0   20.0
            Third   5169.0   15.0

In [36]:
multi_df = df.groupby(['embark_town', 'class'], as_index=False).agg({'fare': ['sum', 'mean']}).round(0)
multi_df

embark_town   class    fare       
                           sum   mean
0    Cherbourg   First  8901.0  105.0
1    Cherbourg  Second   431.0   25.0
2    Cherbourg   Third   740.0   11.0
3   Queenstown   First   180.0   90.0
4   Queenstown  Second    37.0   12.0
5   Queenstown   Third   805.0   11.0
6  Southampton   First  8936.0   70.0
7  Southampton  Second  3334.0   20.0
8  Southampton   Third  5169.0   15.0

In [37]:
multi_df.columns

MultiIndex([('embark_town',     ''),
            (      'class',     ''),
            (       'fare',  'sum'),
            (       'fare', 'mean')],
           )

In [38]:
multi_df.columns = ['_'.join(col).rstrip('_') for col in multi_df.columns.values]
multi_df.head()

,embark_town,class,fare_sum,fare_mean
0,Cherbourg,First,8901.0,105.0
1,Cherbourg,Second,431.0,25.0
2,Cherbourg,Third,740.0,11.0
3,Queenstown,First,180.0,90.0
4,Queenstown,Second,37.0,12.0


# Subtotals

In [39]:
import sidetable

In [41]:
df.groupby(['class', 'embark_town', 'sex']).agg({'fare':'sum'}).stb.subtotal()

fare
class       embark_town       sex                                        
First       Cherbourg         female                            4972.5333
                              male                              3928.5417
                              First | Cherbourg - subtotal      8901.0750
            Queenstown        female                              90.0000
                              male                                90.0000
                              First | Queenstown - subtotal      180.0000
            Southampton       female                            4753.2917
                              male                              4183.0458
                              First | Southampton - subtotal    8936.3375
            First - subtotal                                   18017.4125
Second      Cherbourg         female                             176.8792
                              male                               254.2125
                              Second | Cherbourg - subtotal      431.0917
            Queenstown        female                              24.7000
                              male                                12.3500
                              Second | Queenstown - subtotal      37.0500
            Southampton       female                            1468.1500
                              male                              1865.5500
                              Second | Southampton - subtotal   3333.7000
            Second - subtotal                                   3801.8417
Third       Cherbourg         female                             337.9833
                              male                               402.1462
                              Third | Cherbourg - subtotal       740.1295
            Queenstown        female                             340.1585
                              male                               465.0458
                              Third | Queenstown - subtotal      805.2043
            Southampton       female                            1642.9668
                              male                              3526.3945
                              Third | Southampton - subtotal    5169.3613
            Third - subtotal                                    6714.6951
grand_total                                                    28533.9493